In [56]:
import sqlite3
import enum
import re

import pandas as pd
import dotenv

In [89]:
pd.options.display.max_rows = 8

In [90]:
dotenv.load_dotenv()

True

In [91]:
class StatId(enum.Enum):
    population: str = "0000010101"
    environment: str = "0000010102"
    economics: str = "0000010103"
    administration: str = "0000010104"
    education: str = "0000010105"
    labour: str = "0000010106"
    culture: str = "0000010107"
    housing: str = "0000010108"
    medical_care: str = "0000010109"
    social_security: str = "0000010110"
    household_finance: str = "0000010111"
    daily_routine: str = "0000010112"

In [92]:
def extract_num_from_code(code: str) -> int:
    match_obj = re.search(r"A([0-9]*)", code)
    if match_obj:
        return int(match_obj.group(1))

In [93]:
def get_age_class(code: str) -> int:
    match_obj = re.search(r"A12([0-9]{2})[0-9]{2}", code)
    if match_obj:
        return int(match_obj.group(1)) - 1

In [94]:
def cat01_table(class_data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    cat01_df = class_data["cat01"].copy()
    cat01_df["age_class"] = cat01_df["@code"].apply(get_age_class)
    cat01_df["n"] = cat01_df["@code"].apply(extract_num_from_code)
    cat01_df = cat01_df[cat01_df["n"].between(120101, 122102)]
    cat01_df["sex"] = cat01_df["n"].apply(lambda x: "F" if x % 2 == 0 else "M")
    return cat01_df[["@code", "age_class", "sex"]].astype({"@code": str, "age_class": int, "sex": str})

In [95]:
def area_table(class_data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    area_df = class_data["area"].copy()
    area_df = area_df[area_df["@level"] == "2"].drop(columns = "@level")
    return area_df

In [103]:
def extract_year(year_str: str) -> int:
    match_obj = re.search(r"([0-9]{4}).*", year_str)
    if match_obj:
        return int(match_obj.group(1))

def time_table(class_data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    time_df = class_data["time"].copy()
    time_df["year"] = time_df["@name"].apply(extract_year)
    
    return time_df[["@code", "year"]]

time_table(class_data)

,@code,@name,@level,year
0,1975100000,1975年度,1,1975
1,1976100000,1976年度,1,1976
2,1977100000,1977年度,1,1977
3,1978100000,1978年度,1,1978
...,...,...,...,...
43,2018100000,2018年度,1,2018
44,2019100000,2019年度,1,2019
45,2020100000,2020年度,1,2020
46,2021100000,2021年度,1,2021


In [96]:
conn = sqlite3.connect("./data/japan-stats.db")
record = pd.read_sql("select * from population_record;", conn)

class_data = {
    "tab": pd.read_sql("select * from tab", conn),
    "cat01": pd.read_sql("select * from cat01", conn).drop(columns="@unit"),
    "area": pd.read_sql("select * from area", conn),
    "time": pd.read_sql("select * from time", conn)
}

df = (
    record
    .merge(class_data["tab"], left_on="@tab", right_on="@code", how="left")
    .drop(columns="@code")
    .rename(columns={"@name": "tab", "@level": "tab_level"})
    .merge(class_data["cat01"], left_on="@cat01", right_on="@code", how="left")
    .drop(columns="@code")
    .rename(columns={"@name": "cat01", "@level": "cat01_level"})
    .merge(class_data["area"], left_on="@area", right_on="@code", how="left")
    .drop(columns="@code")
    .rename(columns={"@name": "area", "@level": "area_level"})
    .merge(class_data["time"], left_on="@time", right_on="@code", how="left")
    .drop(columns="@code")
    .rename(columns={"@name": "time", "@level": "time_level"})
)

In [97]:
df

,@tab,@cat01,@area,@time,@unit,$,tab,tab_level,cat01,cat01_level,area,area_level,time,time_level
0,00001,A120101,00000,1975100000,人,5127359,観測値,1,A120101_0～4歳人口（男）,1,全国,1,1975年度,1
1,00001,A120101,00000,1976100000,人,5064000,観測値,1,A120101_0～4歳人口（男）,1,全国,1,1976年度,1
2,00001,A120101,00000,1977100000,人,4929000,観測値,1,A120101_0～4歳人口（男）,1,全国,1,1977年度,1
3,00001,A120101,00000,1978100000,人,4750000,観測値,1,A120101_0～4歳人口（男）,1,全国,1,1978年度,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75740,00001,A122102,47000,2005100000,人,631,観測値,1,A122102_100歳以上人口（女）,1,沖縄県,2,2005年度,1
75741,00001,A122102,47000,2010100000,人,762,観測値,1,A122102_100歳以上人口（女）,1,沖縄県,2,2010年度,1
75742,00001,A122102,47000,2015100000,人,792,観測値,1,A122102_100歳以上人口（女）,1,沖縄県,2,2015年度,1
75743,00001,A122102,47000,2020100000,人,1034,観測値,1,A122102_100歳以上人口（女）,1,沖縄県,2,2020年度,1


In [98]:
cat01_table(class_data)

,@code,age_class,sex
7,A120101,0,M
8,A120102,0,F
10,A120201,1,M
11,A120202,1,F
...,...,...,...
64,A122001,19,M
65,A122002,19,F
67,A122101,20,M
68,A122102,20,F


In [99]:
area_table(class_data)

,@code,@name
1,01000,北海道
2,02000,青森県
3,03000,岩手県
4,04000,宮城県
...,...,...
44,44000,大分県
45,45000,宮崎県
46,46000,鹿児島県
47,47000,沖縄県
